## Import Libraries

In [74]:
import pandas as pd
import re
import emoji
import spacy
# Spacy's english model
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# used to fix contractions such as I'll to I will
import contractions

from bertopic.representation import KeyBERTInspired, PartOfSpeech, MaximalMarginalRelevance
from bertopic import BERTopic

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [75]:
# Filepaths
tweet_data_fp = '../model_evaluation/Results/pruned_labelled_medicine_users.csv'

In [61]:
tweet_data_df = pd.read_csv(tweet_data_fp)
tweet_data_df = tweet_data_df[tweet_data_df['Lang_code'] == 'en']

## Pre-Processing

In [78]:
# Gathering documents
docs = tweet_data_df['Tweet'].tolist()
pruned_docs = docs

In [79]:
len(pruned_docs)

1831

In [80]:
# Best to err on the side of caution when pre-processing:
# https://maartengr.github.io/BERTopic/faq.html#should-i-preprocess-the-data

# Removal of links
def remove_urls(doc):
    return re.sub(r'http\S+', '', doc)

pruned_docs = [remove_urls(doc) for doc in pruned_docs]

# Converting hashtags into words
# Or removing the hashtag and word entirely
# Hashtags are converted to standard words as part of the tokenizer
def remove_hashtags(doc):
    return doc.replace('#', '')
    #return re.sub(r'#\w+', '', doc)

pruned_docs = [remove_hashtags(doc) for doc in pruned_docs]

# Converting mentions into words
def remove_numbers(doc):
    return re.sub(r'\d+', '', doc)

pruned_docs = [remove_numbers(doc) for doc in pruned_docs]

# Function to remove user mentions
def remove_user_mentions(doc):
    return re.sub(r'@\w+', '', doc)

pruned_docs = [remove_user_mentions(doc) for doc in pruned_docs]

def fix_contractions(doc):
    return contractions.fix(doc)

pruned_docs = [fix_contractions(doc) for doc in pruned_docs]

# Remove punctuation 
# Punctuation is removed as part of the tokenizer
def remove_punctuation(doc):
    return re.sub(r'[^\w\s]', '', doc)

pruned_docs = [remove_punctuation(doc) for doc in pruned_docs]

# Function to remove stopwords
def remove_stopwords(doc):
    return ' '.join([word for word in doc.split() if word.lower() not in stop_words])

pruned_docs = [remove_stopwords(doc) for doc in pruned_docs]

# Lemmatize the text
def lemmatize_text(doc):
    doc = nlp(doc)
    return ' '.join([token.lemma_ for token in doc if token.lemma_ not in stop_words and not token.is_punct and not token.is_space])

pruned_docs = [lemmatize_text(doc) for doc in pruned_docs]

# Remove the 'amp' word
def remove_amp(doc):
    return re.sub(r'\bamp\b', '', doc).strip() # strip removes the surrounding white space

pruned_docs = [remove_amp(doc) for doc in pruned_docs]

In [81]:
tokenizer = lambda s: re.findall( '\w+', s.lower() )

pruned_docs_split = [ tokenizer(doc) for doc in  pruned_docs ]

# Combine the inner lists into sentences
pruned_docs_tokenized = [' '.join(words) for words in pruned_docs_split]

## Performing BERTopic

In [82]:
# Topic modelling
bertopic_model = BERTopic()
topics, probs = bertopic_model.fit_transform(pruned_docs_tokenized)

In [83]:
pruned_topic_df = bertopic_model.get_document_info(pruned_docs)

## Calculating Topic Coherence

In [84]:
topic_info = bertopic_model.get_topic_info()
topics = topic_info['Representation'].tolist()

word2id = Dictionary(pruned_docs_split)

In [85]:
cm = CoherenceModel(topics=topics, 
                    texts=pruned_docs_split,
                    coherence='c_npmi',  # c_npmi was used in the bertopic serbian research paper
                    dictionary=word2id)

In [86]:
coherence_per_topic = cm.get_coherence_per_topic()
macro_topic_coherence = sum(coherence_per_topic) / len(coherence_per_topic)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [87]:
# This returns 0.02 when set to npmi, the serbian research return -0.042 as their best score
# This returns 0.511 when set to c_v, this is a good coherence score
# When removing non-english words and links this drops to 0.44
# When adding lemmatization this goes up to 0.46
# Removing numbers drop to 0.4567
macro_topic_coherence 

-0.1039039133490847

In [88]:
print(len(topics))

29


In [89]:
len(topics)

29

## Calculating Topic Diversity

In [90]:
topics

[['covid',
  'vaccine',
  'say',
  'people',
  'pandemic',
  'new',
  'health',
  'coronavirus',
  'get',
  'case'],
 ['china',
  'origin',
  'wuhan',
  'virus',
  'mission',
  'theory',
  'chinese',
  'lab',
  'outbreak',
  'team'],
 ['maryland',
  'smartnews',
  'county',
  'hogan',
  'montgomery',
  'md',
  'gov',
  'patch',
  'case',
  'covid'],
 ['israel',
  'israeli',
  'vaccination',
  'covid',
  'israelis',
  'israels',
  'vaccine',
  'pfizer',
  'show',
  'study'],
 ['vaccine',
  'country',
  'donation',
  'dose',
  'donate',
  'supply',
  'million',
  'vaccination',
  'rich',
  'us'],
 ['vaccine',
  'covid',
  'thread',
  'covax',
  'today',
  'good',
  'interesting',
  'moderna',
  'story',
  'mrna'],
 ['briefing',
  'press',
  'covid',
  'journalist',
  'americas',
  'america',
  'cover',
  'watch',
  'update',
  'situation'],
 ['latin',
  'caribbean',
  'america',
  'paho',
  'vaccine',
  'country',
  'region',
  'director',
  'etienne',
  'brazil'],
 ['thread',
  'good',


In [91]:
from itertools import combinations
def pairwise_jaccard_diversity(topics, topk=10):
    '''
    compute the average pairwise jaccard distance between the topics 
  
    Parameters
    ----------
    topics: a list of lists of words
    topk: top k words on which the topic diversity
          will be computed (There are 10 words in each topic)
    
    Returns
    -------
    pjd: average pairwise jaccard distance
    '''
    dist = 0
    count = 0
    for list1, list2 in combinations(topics, 2):
        js = 1 - len(set(list1).intersection(set(list2)))/len(set(list1).union(set(list2)))
        dist = dist + js
        count = count + 1
    return dist/count

In [92]:
pairwise_jaccard_diversity(topics, 10)

0.9844786069297662

## Adding Topics to the Dataframe

In [118]:
labelled_tweet_data_df = tweet_data_df.copy()

In [119]:
labelled_tweet_data_df['topic_number'] = pruned_topic_df['Topic']
labelled_tweet_data_df['topic_name'] = pruned_topic_df['Name']
labelled_tweet_data_df['topic_words'] = pruned_topic_df['Top_n_words']
labelled_tweet_data_df['topic_probabilities'] = pruned_topic_df['Probability']

In [121]:
labelled_tweet_data_df.to_csv('Results/pruned_topic_labelled_media_users.csv', index=False)